In [17]:
import numpy as np
import pandas as pd
from collections import Counter


In [2]:
facebook_ads = pd.read_csv("2024_fb_ads_president_scored_anon.csv")

In [3]:
import pandas as pd
import json
import ast

# Script to find dictionary/JSON columns in Facebook Ads dataset
print("=== FINDING DICTIONARY COLUMNS ===")

# Get all object columns
object_columns = facebook_ads.select_dtypes(include=['object']).columns
print(f"Object columns to check: {list(object_columns)}")

def check_for_dictionaries(df, column):
    """Check if a column contains dictionaries or JSON structures"""
    print(f"\n--- {column} ---")
    
    # Get first 3 non-null values
    sample_values = df[column].dropna().head(3).tolist()
    
    has_dicts = False
    has_lists = False
    
    for i, val in enumerate(sample_values):
        print(f"Sample {i+1}: {str(val)[:100]}...")
        print(f"Type: {type(val).__name__}")
        
        if isinstance(val, str):
            # Check if it looks like JSON
            if val.strip().startswith('{'):
                try:
                    parsed = json.loads(val)
                    if isinstance(parsed, dict):
                        has_dicts = True
                        print(f"  DICTIONARY detected with keys: {list(parsed.keys())}")
                except:
                    try:
                        parsed = ast.literal_eval(val)
                        if isinstance(parsed, dict):
                            has_dicts = True
                            print(f"  PYTHON DICT detected with keys: {list(parsed.keys())}")
                    except:
                        print("  Looks like dict but can't parse")
            elif val.strip().startswith('['):
                try:
                    parsed = json.loads(val)
                    if isinstance(parsed, list):
                        has_lists = True
                        print(f"  LIST detected with {len(parsed)} items")
                except:
                    try:
                        parsed = ast.literal_eval(val)
                        if isinstance(parsed, list):
                            has_lists = True
                            print(f"  PYTHON LIST detected with {len(parsed)} items")
                    except:
                        print("  Looks like list but can't parse")
            else:
                print("  Regular string")
    
    if has_dicts:
        print("  CONTAINS DICTIONARIES")
        return "DICTIONARY"
    elif has_lists:
        print("  CONTAINS LISTS")
        return "LIST"
    else:
        print("  Just regular strings")
        return "STRING"

# Check each object column
print("\nChecking each object column for complex data structures:")

results = {}
for col in object_columns:
    result = check_for_dictionaries(facebook_ads, col)
    results[col] = result

# Summary
print("\n" + "=" * 50)
print("SUMMARY")
print("=" * 50)

dict_columns = [k for k, v in results.items() if v == "DICTIONARY"]
list_columns = [k for k, v in results.items() if v == "LIST"]
string_columns = [k for k, v in results.items() if v == "STRING"]

print(f"Dictionary columns: {dict_columns}")
print(f"List columns: {list_columns}")
print(f"Simple string columns: {string_columns}")

print(f"\nTotal complex columns to handle: {len(dict_columns) + len(list_columns)}")

=== FINDING DICTIONARY COLUMNS ===
Object columns to check: ['page_id', 'ad_id', 'ad_creation_time', 'bylines', 'currency', 'delivery_by_region', 'demographic_distribution', 'publisher_platforms', 'illuminating_scored_message', 'illuminating_mentions']

Checking each object column for complex data structures:

--- page_id ---
Sample 1: 4ff23a48b53d988df50ddfebb0e442a984ab8f94e874ef9b9cb34394e0c5d230...
Type: str
  Regular string
Sample 2: 4ff23a48b53d988df50ddfebb0e442a984ab8f94e874ef9b9cb34394e0c5d230...
Type: str
  Regular string
Sample 3: 4ff23a48b53d988df50ddfebb0e442a984ab8f94e874ef9b9cb34394e0c5d230...
Type: str
  Regular string
  Just regular strings

--- ad_id ---
Sample 1: 0ddb025b8544e2d58e6977ad417e742a52522b3e1fc1c9d9b61c57148f8d72fc...
Type: str
  Regular string
Sample 2: 86229868e6bde3661724fe02da93504bb4fb5da8c2550d7b7cf193c687e89fa6...
Type: str
  Regular string
Sample 3: 07b5aefc27e872e971f793e49aac38496fa62e484f3928e2b6a2b6e3e08cac8d...
Type: str
  Regular string
  Ju

In [4]:
# After running your detection script...
dict_columns = ['delivery_by_region', 'demographic_distribution']
list_columns = ['publisher_platforms', 'illuminating_mentions']

The below code is designed to convert columns that contain stringified (text) representations of Python dictionaries or lists into real, usable Python objects in the DataFrame.

In [5]:
import json
import ast

def safe_parse(val):
    """Convert a stringified dict/list to real Python object, or leave as-is if not possible."""
    if isinstance(val, str) and val.strip() and (val.strip().startswith("{") or val.strip().startswith("[")):
        try:
            return json.loads(val)
        except Exception:
            try:
                return ast.literal_eval(val)
            except Exception:
                return val  # return original if can't parse
    else:
        return val

for col in dict_columns + list_columns:
    facebook_ads[col] = facebook_ads[col].apply(safe_parse)

I am trying to unpack the delivery_by_region dictionary column so that each (ad, region) combination becomes its own row, with associated spend and impressions.

In [6]:
delivery_rows = []
for idx, row in facebook_ads.iterrows():
    ad_id = row['ad_id']
    d = row['delivery_by_region']
    if isinstance(d, dict):
        for region, vals in d.items():
            delivery_rows.append({
                'ad_id': ad_id,
                'region': region,
                'spend': vals.get('spend'),
                'impressions': vals.get('impressions')
            })
delivery_df = pd.DataFrame(delivery_rows)


In [7]:
demo_rows = []
for idx, row in facebook_ads.iterrows():
    ad_id = row['ad_id']
    d = row['demographic_distribution']
    if isinstance(d, dict):
        for group, vals in d.items():
            demo_rows.append({
                'ad_id': ad_id,
                'group': group,
                'spend': vals.get('spend'),
                'impressions': vals.get('impressions')
            })
demo_df = pd.DataFrame(demo_rows)


In [8]:
facebook_ads['publisher_platforms'] = facebook_ads['publisher_platforms'].apply(
    lambda x: x if isinstance(x, list) else []
)
platform_df = facebook_ads[['ad_id', 'publisher_platforms']].explode('publisher_platforms').rename(
    columns={'publisher_platforms': 'platform'}
)


In [9]:
facebook_ads['illuminating_mentions'] = facebook_ads['illuminating_mentions'].apply(
    lambda x: x if isinstance(x, list) else []
)
mentions_df = facebook_ads[['ad_id', 'illuminating_mentions']].explode('illuminating_mentions').rename(
    columns={'illuminating_mentions': 'mention'}
)

I analysed all the complex "dict/list" columns seperately? 

**Why Analyze Exploded DataFrames Separately?**

The original facebook_ads DataFrame has one row per ad, but columns like delivery_by_region, demographic_distribution, publisher_platforms, and illuminating_mentions are lists or dictionaries that capture multiple regions, demographics, platforms, or mentions per ad.

When we “explode” (unpack) these columns, we create new DataFrames where each row is one (ad, subitem) combination—e.g., one region within an ad.

These exploded DataFrames don’t align row-by-row with the original ads. That’s why we analyze them separately—grouping, summarizing, or visualizing their structure is most meaningful on their own.

In [13]:
# Analyze spend and impressions by region
delivery_region_stats = delivery_df.groupby("region")[["spend", "impressions"]].describe()
print("=== Delivery by Region: Spend & Impressions ===")
print(delivery_region_stats)


=== Delivery by Region: Spend & Impressions ===
                                     spend                                     \
                                     count        mean          std  min  25%   
region                                                                          
Alabama                           105247.0   13.342072    84.308521  0.0  0.0   
Alaska                             96215.0    6.139313    78.537885  0.0  0.0   
Arizona                           122331.0   97.405637   695.468544  0.0  1.0   
Arkansas                          102892.0   10.137533    93.058208  0.0  0.0   
California                        110936.0  176.665005  1047.200365  0.0  4.0   
Colorado                          107790.0   31.158373   169.197975  0.0  0.0   
Connecticut                       105438.0   18.534826    98.279632  0.0  0.0   
Delaware                           97511.0    6.065080    33.541108  0.0  0.0   
Florida                           111351.0   87.892170   531.

In [14]:
# Analyze spend and impressions by demographic group
demo_group_stats = demo_df.groupby("group")[["spend", "impressions"]].describe()
print("\n=== Demographic Distribution: Spend & Impressions ===")
print(demo_group_stats)



=== Demographic Distribution: Spend & Impressions ===
                    spend                                                 \
                    count        mean          std  min  25%   50%   75%   
group                                                                      
female_13-17         73.0    0.095890     0.819288  0.0  0.0   0.0   0.0   
female_18-24     171101.0   58.111846   333.876079  0.0  0.0   2.0  14.0   
female_25-34     194891.0  105.672207   551.114122  0.0  1.0   6.0  31.0   
female_35-44     196516.0  123.394299   604.872368  0.0  1.0   6.0  39.0   
female_45-54     204423.0  113.538208   561.588600  0.0  2.0   7.0  40.0   
female_55-64     198293.0  146.581932   751.500109  0.0  4.0  10.0  55.0   
female_65+       196563.0  223.882603  1216.710992  0.0  7.0  18.0  80.0   
female_Unknown        5.0    0.000000     0.000000  0.0  0.0   0.0   0.0   
male_13-17          149.0    0.053691     0.503840  0.0  0.0   0.0   0.0   
male_18-24       172141.0   39.72

In [ ]:
# Count number of ads shown per platform
platform_counts = platform_df["platform"].value_counts()
print("\n=== Platform Distribution ===")
print(platform_counts)


In [ ]:
# Count number of ads mentioning each entity/person
mentions_counts = mentions_df["mention"].value_counts()
print("\n=== Illuminating Mentions Distribution ===")
print(mentions_counts)


In [23]:
import pandas as pd
import time

# Facebook Ads Groupby Analysis - Pandas
print("=== FACEBOOK ADS GROUPBY - PANDAS ===")

start_time = time.time()

# Remove complex columns for clean analysis
exclude_cols = ['delivery_by_region', 'demographic_distribution', 'publisher_platforms', 'illuminating_mentions']
ads_pandas = facebook_ads.drop(columns=[c for c in exclude_cols if c in facebook_ads.columns])

print(f"Dataset: {len(ads_pandas):,} ads, {len(ads_pandas.columns)} columns")
print(f"Grouping by: page_id ({ads_pandas['page_id'].nunique():,} unique pages)")

print("\n1. TOP 10 PAGES BY TOTAL SPEND")
print("=" * 60)

# Group by page_id for spend analysis
spend_by_page = ads_pandas.groupby("page_id")['estimated_spend'].agg(['count', 'sum', 'mean']).round(0)
spend_by_page.columns = ['ad_count', 'total_spend', 'avg_spend']
spend_by_page = spend_by_page.sort_values('total_spend', ascending=False)

print(f"{'Page ID':<50} {'Ads':<6} {'Total Spend':<12} {'Avg Spend':<10}")
print("-" * 80)
for page_id, row in spend_by_page.head(10).iterrows():
    print(f"{page_id[:47]:<50} {row['ad_count']:<6.0f} ${row['total_spend']:<11,.0f} ${row['avg_spend']:<9.0f}")

print("\n2. TOP 10 PAGES BY TOTAL IMPRESSIONS")
print("=" * 60)

# Group by page_id for impressions analysis
impressions_by_page = ads_pandas.groupby("page_id")['estimated_impressions'].agg(['count', 'sum', 'mean']).round(0)
impressions_by_page.columns = ['ad_count', 'total_impressions', 'avg_impressions']
impressions_by_page = impressions_by_page.sort_values('total_impressions', ascending=False)

print(f"{'Page ID':<50} {'Ads':<6} {'Total Impressions':<15} {'Avg Impressions':<12}")
print("-" * 85)
for page_id, row in impressions_by_page.head(10).iterrows():
    print(f"{page_id[:47]:<50} {row['ad_count']:<6.0f} {row['total_impressions']:<15,.0f} {row['avg_impressions']:<12.0f}")

print("\n3. TOP 10 PAGES BY AD COUNT")
print("=" * 60)

# Group by page_id for ad count
ad_count_by_page = ads_pandas.groupby("page_id").size().sort_values(ascending=False)
ad_count_by_page.name = 'ad_count'

print(f"{'Page ID':<50} {'Ad Count':<10}")
print("-" * 62)
for page_id, ad_count in ad_count_by_page.head(10).items():
    print(f"{page_id[:47]:<50} {ad_count:<10}")

print("\n4. TOPIC ANALYSIS BY PAGE")
print("=" * 60)

# Economy topic analysis
economy_by_page = ads_pandas.groupby("page_id")['economy_topic_illuminating'].sum().sort_values(ascending=False)

print("Top 3 pages for economy topic:")
for page_id, count in economy_by_page.head(3).items():
    print(f"  {page_id[:40]}... : {count} ads")

# Health topic analysis
health_by_page = ads_pandas.groupby("page_id")['health_topic_illuminating'].sum().sort_values(ascending=False)

print("\nTop 3 pages for health topic:")
for page_id, count in health_by_page.head(3).items():
    print(f"  {page_id[:40]}... : {count} ads")

print("\n5. BYLINES (CAMPAIGN NAMES) ANALYSIS")
print("=" * 60)

# Analyze campaign name diversity
bylines_stats = ads_pandas.groupby("page_id")['bylines'].agg(['count', 'nunique']).fillna(0)
bylines_stats.columns = ['total_ads', 'campaign_names']
bylines_stats = bylines_stats.sort_values('campaign_names', ascending=False)

print("Pages with most campaign name variations:")
print(f"{'Page ID':<50} {'Total Ads':<10} {'Campaign Names':<15}")
print("-" * 77)
for page_id, row in bylines_stats.head(5).iterrows():
    print(f"{page_id[:47]:<50} {row['total_ads']:<10.0f} {row['campaign_names']:<15.0f}")

# Performance timing
end_time = time.time()
execution_time = end_time - start_time

print(f"\n6. EXECUTION SUMMARY")
print("=" * 60)
print(f"Pandas execution time: {execution_time:.3f} seconds")
print(f"Pages analyzed: {ads_pandas['page_id'].nunique():,}")
print(f"Total ads: {len(ads_pandas):,}")

# Save results for comparison
pandas_ads_results = {
    'spend_by_page': spend_by_page,
    'impressions_by_page': impressions_by_page,
    'ad_count_by_page': ad_count_by_page,
    'execution_time': execution_time
}

print("Results saved in 'pandas_ads_results' variable")

=== FACEBOOK ADS GROUPBY - PANDAS ===
Dataset: 246,745 ads, 37 columns
Grouping by: page_id (4,475 unique pages)

1. TOP 10 PAGES BY TOTAL SPEND
Page ID                                            Ads    Total Spend  Avg Spend 
--------------------------------------------------------------------------------
4d66f5853f0365dba032a87704a634f023d15babde973bb    55503  $82,795,647  $1492     
4ade404186269ec62d2dd7d9e0ed5f93a5f32c057516879    14822  $26,367,978  $1779     
e3342051b60393770363ffc02946a0f76bc3e4155190d1f    23988  $19,609,362  $817      
ec8ac6dc1cddc49972de2c31b62343fe3979729ec437c3b    9851   $10,599,999  $1076     
e3ee066f4a12968ba94847059a03c33965ca4eb1ead0f6f    6581   $7,634,469   $1160     
330b2f35ded2161e63fbb2b5c5bdae05bff274ff31f990c    10461  $6,135,389   $587      
3783ccf18c17765d36f5df639d94a17b6ce3f91f2ec886a    1210   $4,267,040   $3526     
66dbd85261d199af2e22a60a56b02e520d2bd66256eb7b6    2203   $4,245,397   $1927     
d46be6146feba33b5fd538ae85ded01b0393

In [21]:
import time
from collections import defaultdict

# Facebook Ads Groupby Analysis - Pure Python
print("=== FACEBOOK ADS GROUPBY - PURE PYTHON ===")

start_time = time.time()

# Convert to list of dictionaries (exclude complex columns)
exclude_cols = ['delivery_by_region', 'demographic_distribution', 'publisher_platforms', 'illuminating_mentions']
ads_data = facebook_ads.drop(columns=[c for c in exclude_cols if c in facebook_ads.columns]).to_dict('records')

print(f"Dataset: {len(ads_data):,} ads")

# Manual groupby using dictionaries
page_groups = defaultdict(list)
for ad in ads_data:
    page_id = ad.get('page_id')
    if page_id:
        page_groups[page_id].append(ad)

print(f"Grouping by: page_id ({len(page_groups):,} unique pages)")

def safe_float(value):
    """Convert value to float safely"""
    try:
        return float(value) if value is not None else 0
    except:
        return 0

def calculate_group_stats(ads_list, metric):
    """Calculate count, sum, mean for a metric across ads"""
    values = [safe_float(ad.get(metric, 0)) for ad in ads_list]
    values = [v for v in values if v > 0]  # Remove zeros for meaningful stats
    
    if not values:
        return {'count': 0, 'sum': 0, 'mean': 0}
    
    return {
        'count': len(values),
        'sum': sum(values),
        'mean': sum(values) / len(values)
    }

print("\n1. TOP 10 PAGES BY TOTAL SPEND")
print("=" * 60)

# Calculate spend stats for each page
spend_stats = {}
for page_id, ads_list in page_groups.items():
    spend_stats[page_id] = calculate_group_stats(ads_list, 'estimated_spend')

# Sort by total spend
sorted_spend = sorted(spend_stats.items(), key=lambda x: x[1]['sum'], reverse=True)

print(f"{'Page ID':<50} {'Ads':<6} {'Total Spend':<12} {'Avg Spend':<10}")
print("-" * 80)
for page_id, stats in sorted_spend[:10]:
    print(f"{page_id[:47]:<50} {stats['count']:<6} ${stats['sum']:<11,.0f} ${stats['mean']:<9.0f}")

print("\n2. TOP 10 PAGES BY TOTAL IMPRESSIONS")
print("=" * 60)

# Calculate impression stats for each page
impression_stats = {}
for page_id, ads_list in page_groups.items():
    impression_stats[page_id] = calculate_group_stats(ads_list, 'estimated_impressions')

# Sort by total impressions
sorted_impressions = sorted(impression_stats.items(), key=lambda x: x[1]['sum'], reverse=True)

print(f"{'Page ID':<50} {'Ads':<6} {'Total Impressions':<15} {'Avg Impressions':<12}")
print("-" * 85)
for page_id, stats in sorted_impressions[:10]:
    print(f"{page_id[:47]:<50} {stats['count']:<6} {stats['sum']:<15,.0f} {stats['mean']:<12.0f}")

print("\n3. TOP 10 PAGES BY AD COUNT")
print("=" * 60)

# Count ads per page
ad_counts = {page_id: len(ads_list) for page_id, ads_list in page_groups.items()}
sorted_counts = sorted(ad_counts.items(), key=lambda x: x[1], reverse=True)

print(f"{'Page ID':<50} {'Ad Count':<10}")
print("-" * 62)
for page_id, count in sorted_counts[:10]:
    print(f"{page_id[:47]:<50} {count:<10}")

print("\n4. TOPIC ANALYSIS BY PAGE")
print("=" * 60)

# Economy topic analysis
economy_by_page = {}
for page_id, ads_list in page_groups.items():
    economy_count = sum(safe_float(ad.get('economy_topic_illuminating', 0)) for ad in ads_list)
    economy_by_page[page_id] = economy_count

sorted_economy = sorted(economy_by_page.items(), key=lambda x: x[1], reverse=True)
print("Top 3 pages for economy topic:")
for page_id, count in sorted_economy[:3]:
    print(f"  {page_id[:40]}... : {count:.0f} ads")

# Health topic analysis
health_by_page = {}
for page_id, ads_list in page_groups.items():
    health_count = sum(safe_float(ad.get('health_topic_illuminating', 0)) for ad in ads_list)
    health_by_page[page_id] = health_count

sorted_health = sorted(health_by_page.items(), key=lambda x: x[1], reverse=True)
print("\nTop 3 pages for health topic:")
for page_id, count in sorted_health[:3]:
    print(f"  {page_id[:40]}... : {count:.0f} ads")

print("\n5. BYLINES (CAMPAIGN NAMES) ANALYSIS")
print("=" * 60)

# Analyze campaign name diversity
bylines_stats = {}
for page_id, ads_list in page_groups.items():
    bylines = [ad.get('bylines') for ad in ads_list if ad.get('bylines')]
    unique_bylines = set(bylines) if bylines else set()
    
    bylines_stats[page_id] = {
        'total_ads': len(ads_list),
        'campaign_names': len(unique_bylines)
    }

# Sort by campaign name diversity
sorted_bylines = sorted(bylines_stats.items(), key=lambda x: x[1]['campaign_names'], reverse=True)

print("Pages with most campaign name variations:")
print(f"{'Page ID':<50} {'Total Ads':<10} {'Campaign Names':<15}")
print("-" * 77)
for page_id, stats in sorted_bylines[:5]:
    print(f"{page_id[:47]:<50} {stats['total_ads']:<10} {stats['campaign_names']:<15}")

# Performance timing
end_time = time.time()
execution_time = end_time - start_time

print(f"\n6. EXECUTION SUMMARY")
print("=" * 60)
print(f"Pure Python execution time: {execution_time:.3f} seconds")
print(f"Pages analyzed: {len(page_groups):,}")
print(f"Total ads: {len(ads_data):,}")

# Save results for comparison
python_ads_results = {
    'spend_stats': dict(sorted_spend),
    'impression_stats': dict(sorted_impressions),
    'ad_counts': dict(sorted_counts),
    'execution_time': execution_time
}

print("Results saved in 'python_ads_results' variable")

=== FACEBOOK ADS GROUPBY - PURE PYTHON ===
Dataset: 246,745 ads
Grouping by: page_id (4,475 unique pages)

1. TOP 10 PAGES BY TOTAL SPEND
Page ID                                            Ads    Total Spend  Avg Spend 
--------------------------------------------------------------------------------
4d66f5853f0365dba032a87704a634f023d15babde973bb    55503  $82,795,647  $1492     
4ade404186269ec62d2dd7d9e0ed5f93a5f32c057516879    14822  $26,367,978  $1779     
e3342051b60393770363ffc02946a0f76bc3e4155190d1f    23988  $19,609,362  $817      
ec8ac6dc1cddc49972de2c31b62343fe3979729ec437c3b    9851   $10,599,999  $1076     
e3ee066f4a12968ba94847059a03c33965ca4eb1ead0f6f    6581   $7,634,469   $1160     
330b2f35ded2161e63fbb2b5c5bdae05bff274ff31f990c    10461  $6,135,389   $587      
3783ccf18c17765d36f5df639d94a17b6ce3f91f2ec886a    1210   $4,267,040   $3526     
66dbd85261d199af2e22a60a56b02e520d2bd66256eb7b6    2203   $4,245,397   $1927     
d46be6146feba33b5fd538ae85ded01b0393c7ee2c1

In [22]:
import polars as pl
import time

# Facebook Ads Groupby Analysis - Polars
print("=== FACEBOOK ADS GROUPBY - POLARS ===")

start_time = time.time()

# Convert pandas DataFrame to Polars and remove complex columns
exclude_cols = ['delivery_by_region', 'demographic_distribution', 'publisher_platforms', 'illuminating_mentions']
ads_clean_df = facebook_ads.drop(columns=[c for c in exclude_cols if c in facebook_ads.columns])

# Convert to Polars
try:
    ads_polars = pl.from_pandas(ads_clean_df)
    print("Successfully converted to Polars DataFrame")
except Exception as e:
    print(f"Conversion error: {e}")
    print("Using alternative method...")
    # Alternative: save to CSV and reload with Polars
    ads_clean_df.to_csv('temp_ads.csv', index=False)
    ads_polars = pl.read_csv('temp_ads.csv')

print(f"Dataset: {ads_polars.height:,} ads, {len(ads_polars.columns)} columns")
print(f"Grouping by: page_id ({ads_polars.select('page_id').n_unique():,} unique pages)")

print("\n1. TOP 10 PAGES BY TOTAL SPEND")
print("=" * 60)

# Group by page_id for spend analysis
spend_by_page = ads_polars.group_by("page_id").agg([
    pl.col('estimated_spend').count().alias('ad_count'),
    pl.col('estimated_spend').sum().alias('total_spend'),
    pl.col('estimated_spend').mean().alias('avg_spend')
]).sort('total_spend', descending=True)

print(f"{'Page ID':<50} {'Ads':<6} {'Total Spend':<12} {'Avg Spend':<10}")
print("-" * 80)
for i in range(min(10, spend_by_page.height)):
    page_id = spend_by_page.item(i, 'page_id')
    ad_count = spend_by_page.item(i, 'ad_count')
    total_spend = spend_by_page.item(i, 'total_spend')
    avg_spend = spend_by_page.item(i, 'avg_spend')
    
    print(f"{page_id[:47]:<50} {ad_count:<6} ${total_spend:<11,.0f} ${avg_spend:<9.0f}")

print("\n2. TOP 10 PAGES BY TOTAL IMPRESSIONS")
print("=" * 60)

# Group by page_id for impressions analysis
impressions_by_page = ads_polars.group_by("page_id").agg([
    pl.col('estimated_impressions').count().alias('ad_count'),
    pl.col('estimated_impressions').sum().alias('total_impressions'),
    pl.col('estimated_impressions').mean().alias('avg_impressions')
]).sort('total_impressions', descending=True)

print(f"{'Page ID':<50} {'Ads':<6} {'Total Impressions':<15} {'Avg Impressions':<12}")
print("-" * 85)
for i in range(min(10, impressions_by_page.height)):
    page_id = impressions_by_page.item(i, 'page_id')
    ad_count = impressions_by_page.item(i, 'ad_count')
    total_impressions = impressions_by_page.item(i, 'total_impressions')
    avg_impressions = impressions_by_page.item(i, 'avg_impressions')
    
    print(f"{page_id[:47]:<50} {ad_count:<6} {total_impressions:<15,.0f} {avg_impressions:<12.0f}")

print("\n3. TOP 10 PAGES BY AD COUNT")
print("=" * 60)

# Group by page_id for ad count
ad_count_by_page = ads_polars.group_by("page_id").agg([
    pl.len().alias('ad_count')
]).sort('ad_count', descending=True)

print(f"{'Page ID':<50} {'Ad Count':<10}")
print("-" * 62)
for i in range(min(10, ad_count_by_page.height)):
    page_id = ad_count_by_page.item(i, 'page_id')
    ad_count = ad_count_by_page.item(i, 'ad_count')
    print(f"{page_id[:47]:<50} {ad_count:<10}")

print("\n4. TOPIC ANALYSIS BY PAGE")
print("=" * 60)

# Economy topic analysis
economy_by_page = ads_polars.group_by("page_id").agg([
    pl.col('economy_topic_illuminating').sum().alias('economy_ads')
]).sort('economy_ads', descending=True)

print("Top 3 pages for economy topic:")
for i in range(min(3, economy_by_page.height)):
    page_id = economy_by_page.item(i, 'page_id')
    count = economy_by_page.item(i, 'economy_ads')
    print(f"  {page_id[:40]}... : {count} ads")

# Health topic analysis
health_by_page = ads_polars.group_by("page_id").agg([
    pl.col('health_topic_illuminating').sum().alias('health_ads')
]).sort('health_ads', descending=True)

print("\nTop 3 pages for health topic:")
for i in range(min(3, health_by_page.height)):
    page_id = health_by_page.item(i, 'page_id')
    count = health_by_page.item(i, 'health_ads')
    print(f"  {page_id[:40]}... : {count} ads")

print("\n5. BYLINES (CAMPAIGN NAMES) ANALYSIS")
print("=" * 60)

# Analyze campaign name diversity
bylines_stats = ads_polars.group_by("page_id").agg([
    pl.len().alias('total_ads'),
    pl.col('bylines').n_unique().alias('campaign_names')
]).sort('campaign_names', descending=True)

print("Pages with most campaign name variations:")
print(f"{'Page ID':<50} {'Total Ads':<10} {'Campaign Names':<15}")
print("-" * 77)
for i in range(min(5, bylines_stats.height)):
    page_id = bylines_stats.item(i, 'page_id')
    total_ads = bylines_stats.item(i, 'total_ads')
    campaign_names = bylines_stats.item(i, 'campaign_names')
    print(f"{page_id[:47]:<50} {total_ads:<10} {campaign_names:<15}")

# Performance timing
end_time = time.time()
execution_time = end_time - start_time

print(f"\n6. EXECUTION SUMMARY")
print("=" * 60)
print(f"Polars execution time: {execution_time:.3f} seconds")
print(f"Pages analyzed: {ads_polars.select('page_id').n_unique():,}")
print(f"Total ads: {ads_polars.height:,}")

# Save results for comparison
polars_ads_results = {
    'spend_by_page': spend_by_page,
    'impressions_by_page': impressions_by_page,
    'ad_count_by_page': ad_count_by_page,
    'execution_time': execution_time
}

print("Results saved in 'polars_ads_results' variable")

=== FACEBOOK ADS GROUPBY - POLARS ===
Successfully converted to Polars DataFrame
Dataset: 246,745 ads, 37 columns
Grouping by: page_id (4,475 unique pages)

1. TOP 10 PAGES BY TOTAL SPEND
Page ID                                            Ads    Total Spend  Avg Spend 
--------------------------------------------------------------------------------
4d66f5853f0365dba032a87704a634f023d15babde973bb    55503  $82,795,647  $1492     
4ade404186269ec62d2dd7d9e0ed5f93a5f32c057516879    14822  $26,367,978  $1779     
e3342051b60393770363ffc02946a0f76bc3e4155190d1f    23988  $19,609,362  $817      
ec8ac6dc1cddc49972de2c31b62343fe3979729ec437c3b    9851   $10,599,999  $1076     
e3ee066f4a12968ba94847059a03c33965ca4eb1ead0f6f    6581   $7,634,469   $1160     
330b2f35ded2161e63fbb2b5c5bdae05bff274ff31f990c    10461  $6,135,389   $587      
3783ccf18c17765d36f5df639d94a17b6ce3f91f2ec886a    1210   $4,267,040   $3526     
66dbd85261d199af2e22a60a56b02e520d2bd66256eb7b6    2203   $4,245,397   $192

## My Comparison: Polars vs. Pandas vs. Pure Python GroupBy**

**1. What I Did and Why**

I wanted to analyze my Facebook Ads data by page (page_id) to answer questions like:

Which pages spent the most?

Which pages got the most impressions?

Which pages ran the most ads?

To be thorough, I did this using three different approaches: Polars, Pandas, and pure Python, so I could compare speed, convenience, and results.

2. What I Found
All three methods gave me the same answers for the big questions (top pages by spend, impressions, ad count, top topics, and campaign name diversity).

Polars was the fastest—done in about 0.25 seconds!

Pandas was also quick and is easy to use for most data analysis tasks.

Pure Python worked, but was much slower and would be a pain with huge datasets.

3. Why I Handled Complex Columns Separately
Some columns—like delivery_by_region, demographic_distribution, and illuminating_mentions—aren’t simple numbers or labels. They’re actually lists or dictionaries inside each row.

I didn’t include these in my main groupby summaries (like mean, sum, etc.) because they don’t make sense for those calculations. If I did, I’d get unreadable outputs (huge blobs of text or JSON).

Instead, I unpacked or exploded these columns and analyzed them one at a time. For example:

I might count how many times a candidate was mentioned across all ads.

Or I could summarize spend by state if the region info is nested.

This kept my groupby tables clear, focused, and readable.

4. Why I Took This Approach
I grouped straightforward columns together for classic stats and rankings.

For the complex or nested columns, I wrote special logic to get real insights, not just noise.

5. Which Tool Would I Use (and Why)?
For very large datasets, I’d choose Polars—it’s lightning fast.

For most analysis tasks, I’d go with Pandas for its power and ease of use.

I’d only use pure Python if I absolutely had to (e.g., no external packages allowed).

Summary
I used three different tools to make sure my results were consistent and reliable.
I analyzed complex/nested columns separately so my groupby stats stayed clear and actionable.
This approach kept my analysis fast, tidy, and actually useful.

